In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from crowdnalysis.cmdstan import StanMultinomialOptimizeConsensus, StanMultinomial2OptimizeConsensus
from crowdnalysis.dawid_skene import DawidSkene
import numpy as np
import json

In [8]:
smoc = StanMultinomialOptimizeConsensus()
ds = DawidSkene()

In [40]:
t = 100
num_annotations_per_task = 5
real_tau = np.array([0.1, 0.3, 0.6])
k = len(real_tau)
real_pi = np.array([[0.5, 0.3, 0.2], [0.2, 0.6, 0.2], [0.25, 0.3, 0.45]]) 
print(real_pi)
t_C, m = smoc.sample(t, num_annotations_per_task, parameters = {'tau':real_tau, 'pi':real_pi})

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.sample_tasks
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.sample_annotations
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


[[0.5  0.3  0.2 ]
 [0.2  0.6  0.2 ]
 [0.25 0.3  0.45]]


In [41]:
smoc.model_name

'Multinomial'

In [42]:
m

array([[ 0,  0,  1],
       [ 0,  0,  2],
       [ 0,  0,  1],
       ...,
       [99,  0,  1],
       [99,  0,  1],
       [99,  0,  1]])

In [43]:
t_A = m[:,0]+1
w_A = m[:,1]
ann = m[:,2]+1
w = 20
#t = np.unique(t_A).shape[0]
a = m.shape[0]
for t_ in range(t):
    w_A[t_*num_annotations_per_task:(t_+1)*num_annotations_per_task] = \
        np.random.choice(w, size=num_annotations_per_task, replace=False)
#w_A += 1
#w_A = np.random.choice(w, size=10)
json_data = {'w': w,
             't': t,
             'a': a,
             'k': k,
             't_A': t_A.tolist(),
             'w_A': w_A.tolist(),
             'ann': ann.tolist()}

In [44]:
print("Max w_A:",np.max(w_A))
with open('multinomial.json', 'w') as outfile:
    json.dump(json_data, outfile, indent=2)

Max w_A: 19


In [45]:
print(a)
print(m[9980:10000], t_A[9980:])

500
[] []


In [46]:

sm2oc = StanMultinomial2OptimizeConsensus()
t_C, d = sm2oc.m_fit_and_compute_consensus(m, t, k, w)
d

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial2.fit_and_consensus
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


{'w': 20, 't': 100, 'a': 500, 'k': 3, 't_A': array([  1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   3,   3,   3,
         3,   3,   4,   4,   4,   4,   4,   5,   5,   5,   5,   5,   6,
         6,   6,   6,   6,   7,   7,   7,   7,   7,   8,   8,   8,   8,
         8,   9,   9,   9,   9,   9,  10,  10,  10,  10,  10,  11,  11,
        11,  11,  11,  12,  12,  12,  12,  12,  13,  13,  13,  13,  13,
        14,  14,  14,  14,  14,  15,  15,  15,  15,  15,  16,  16,  16,
        16,  16,  17,  17,  17,  17,  17,  18,  18,  18,  18,  18,  19,
        19,  19,  19,  19,  20,  20,  20,  20,  20,  21,  21,  21,  21,
        21,  22,  22,  22,  22,  22,  23,  23,  23,  23,  23,  24,  24,
        24,  24,  24,  25,  25,  25,  25,  25,  26,  26,  26,  26,  26,
        27,  27,  27,  27,  27,  28,  28,  28,  28,  28,  29,  29,  29,
        29,  29,  30,  30,  30,  30,  30,  31,  31,  31,  31,  31,  32,
        32,  32,  32,  32,  33,  33,  33,  33,  33,  34,  34,  34,  34,
        34,  35,  3

{'tau': array([0.129318, 0.154011, 0.71667 ]),
 'eta': array([[5.55105e-05, 4.63835e-01],
        [2.72217e+00, 1.87734e+00],
        [6.55009e-01, 4.25918e-01]]),
 'pi': array([[0.3804   , 0.380379 , 0.239221 ],
        [0.0539348, 0.820527 , 0.125538 ],
        [0.239085 , 0.300639 , 0.460276 ]]),
 'log_p_t_C': array([[ -7.80592,  -9.33661,  -5.71962],
        [ -7.34214,  -7.45928,  -6.14554],
        [ -8.73353, -10.3691 ,  -4.6387 ],
        [ -8.2697 , -11.2139 ,  -5.29371],
        [ -7.80586, -12.0588 ,  -5.94872],
        [ -7.80592,  -9.33661,  -5.71962],
        [ -6.87825, -11.0263 ,  -7.02964],
        [ -7.80592,  -9.33661,  -5.71962],
        [ -8.26964, -13.9361 ,  -5.5228 ],
        [ -7.80592,  -9.33661,  -5.71962],
        [ -7.34208, -10.1814 ,  -6.37463],
        [ -7.34203, -12.9036 ,  -6.60372],
        [ -6.87825, -11.0263 ,  -7.02964],
        [ -7.80597,  -6.61444,  -5.49053],
        [ -7.80592,  -9.33661,  -5.71962],
        [ -6.8783 ,  -8.30411,  -6.80055]

In [47]:
t_C, d = smoc.m_fit_and_compute_consensus(m, t, k, w)
d

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.fit_and_consensus
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


{'w': 20, 't': 100, 'a': 500, 'k': 3, 't_A': array([  1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   3,   3,   3,
         3,   3,   4,   4,   4,   4,   4,   5,   5,   5,   5,   5,   6,
         6,   6,   6,   6,   7,   7,   7,   7,   7,   8,   8,   8,   8,
         8,   9,   9,   9,   9,   9,  10,  10,  10,  10,  10,  11,  11,
        11,  11,  11,  12,  12,  12,  12,  12,  13,  13,  13,  13,  13,
        14,  14,  14,  14,  14,  15,  15,  15,  15,  15,  16,  16,  16,
        16,  16,  17,  17,  17,  17,  17,  18,  18,  18,  18,  18,  19,
        19,  19,  19,  19,  20,  20,  20,  20,  20,  21,  21,  21,  21,
        21,  22,  22,  22,  22,  22,  23,  23,  23,  23,  23,  24,  24,
        24,  24,  24,  25,  25,  25,  25,  25,  26,  26,  26,  26,  26,
        27,  27,  27,  27,  27,  28,  28,  28,  28,  28,  29,  29,  29,
        29,  29,  30,  30,  30,  30,  30,  31,  31,  31,  31,  31,  32,
        32,  32,  32,  32,  33,  33,  33,  33,  33,  34,  34,  34,  34,
        34,  35,  3

{'tau': array([0.00143173, 0.104611  , 0.893957  ]),
 'pi': array([[9.99983e-01, 1.51337e-05, 1.89846e-06],
        [1.14253e-02, 9.37018e-01, 5.15567e-02],
        [2.40169e-01, 3.19727e-01, 4.40104e-01]]),
 'log_p_t_C': array([[-55.095  , -12.7897 ,  -5.46057],
        [-53.0191 ,  -9.88966,  -5.78012],
        [-70.3453 , -14.1828 ,  -4.53536],
        [-57.1709 , -15.6897 ,  -5.14103],
        [-43.9964 , -17.1966 ,  -5.7467 ],
        [-55.095  , -12.7897 ,  -5.46057],
        [-28.7461 , -15.8034 ,  -6.67192],
        [-55.095  , -12.7897 ,  -5.46057],
        [-46.0723 , -20.0966 ,  -5.42716],
        [-55.095  , -12.7897 ,  -5.46057],
        [-41.9206 , -14.2965 ,  -6.06625],
        [-30.822  , -18.7034 ,  -6.35237],
        [-28.7461 , -15.8034 ,  -6.67192],
        [-66.1936 ,  -8.38281,  -5.17445],
        [-55.095  , -12.7897 ,  -5.46057],
        [-39.8447 , -11.3965 ,  -6.38579],
        [-64.1177 ,  -5.48279,  -5.49399],
        [-64.1177 ,  -5.48279,  -5.49399],
     

In [48]:
m[:,1]=0
a = ds.m_fit_and_compute_consensus(m, t, k, 1)
a


(array([[0.47629916, 0.07265595, 0.45104489],
        [0.49253281, 0.30742989, 0.2000373 ],
        [0.18053211, 0.01019056, 0.80927733],
        [0.30813743, 0.01148725, 0.68037531],
        [0.47343758, 0.01165636, 0.51490606],
        [0.47629916, 0.07265595, 0.45104489],
        [0.77143334, 0.05132708, 0.17723958],
        [0.47629916, 0.07265595, 0.45104489],
        [0.28233425, 0.00169881, 0.71596694],
        [0.47629916, 0.07265595, 0.45104489],
        [0.63808478, 0.0642833 , 0.29763192],
        [0.64434392, 0.01047723, 0.34517886],
        [0.77143334, 0.05132708, 0.17723958],
        [0.36105569, 0.34123747, 0.29770684],
        [0.47629916, 0.07265595, 0.45104489],
        [0.62024004, 0.25568139, 0.12407857],
        [0.1915386 , 0.74072739, 0.06773401],
        [0.1915386 , 0.74072739, 0.06773401],
        [0.36105569, 0.34123747, 0.29770684],
        [0.47343758, 0.01165636, 0.51490606],
        [0.30813743, 0.01148725, 0.68037531],
        [0.64434392, 0.01047723, 0

In [49]:
d = smoc.m_fit(m, t, k, w, t_C)
d

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.fit
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


{'w': 20, 't': 100, 'a': 500, 'k': 3, 't_A': array([  1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   3,   3,   3,
         3,   3,   4,   4,   4,   4,   4,   5,   5,   5,   5,   5,   6,
         6,   6,   6,   6,   7,   7,   7,   7,   7,   8,   8,   8,   8,
         8,   9,   9,   9,   9,   9,  10,  10,  10,  10,  10,  11,  11,
        11,  11,  11,  12,  12,  12,  12,  12,  13,  13,  13,  13,  13,
        14,  14,  14,  14,  14,  15,  15,  15,  15,  15,  16,  16,  16,
        16,  16,  17,  17,  17,  17,  17,  18,  18,  18,  18,  18,  19,
        19,  19,  19,  19,  20,  20,  20,  20,  20,  21,  21,  21,  21,
        21,  22,  22,  22,  22,  22,  23,  23,  23,  23,  23,  24,  24,
        24,  24,  24,  25,  25,  25,  25,  25,  26,  26,  26,  26,  26,
        27,  27,  27,  27,  27,  28,  28,  28,  28,  28,  29,  29,  29,
        29,  29,  30,  30,  30,  30,  30,  31,  31,  31,  31,  31,  32,
        32,  32,  32,  32,  33,  33,  33,  33,  33,  34,  34,  34,  34,
        34,  35,  3

{'tau': array([0.00140936, 0.104596  , 0.893995  ]),
 'pi': array([[9.99992e-01, 3.75113e-06, 3.75113e-06],
        [1.14160e-02, 9.36988e-01, 5.15958e-02],
        [2.40188e-01, 3.19697e-01, 4.40114e-01]])}

In [50]:
d2 = smoc.m_compute_consensus(m, t, k, w, data=d)
d2

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.consensus
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


{'w': 20, 't': 100, 'a': 500, 'k': 3, 't_A': array([  1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   3,   3,   3,
         3,   3,   4,   4,   4,   4,   4,   5,   5,   5,   5,   5,   6,
         6,   6,   6,   6,   7,   7,   7,   7,   7,   8,   8,   8,   8,
         8,   9,   9,   9,   9,   9,  10,  10,  10,  10,  10,  11,  11,
        11,  11,  11,  12,  12,  12,  12,  12,  13,  13,  13,  13,  13,
        14,  14,  14,  14,  14,  15,  15,  15,  15,  15,  16,  16,  16,
        16,  16,  17,  17,  17,  17,  17,  18,  18,  18,  18,  18,  19,
        19,  19,  19,  19,  20,  20,  20,  20,  20,  21,  21,  21,  21,
        21,  22,  22,  22,  22,  22,  23,  23,  23,  23,  23,  24,  24,
        24,  24,  24,  25,  25,  25,  25,  25,  26,  26,  26,  26,  26,
        27,  27,  27,  27,  27,  28,  28,  28,  28,  28,  29,  29,  29,
        29,  29,  30,  30,  30,  30,  30,  31,  31,  31,  31,  31,  32,
        32,  32,  32,  32,  33,  33,  33,  33,  33,  34,  34,  34,  34,
        34,  35,  3

(array([[6.55976e-23, 6.56065e-04, 9.99344e-01],
        [8.89057e-23, 1.61476e-02, 9.83852e-01],
        [9.76045e-29, 6.47655e-05, 9.99935e-01],
        [4.76799e-23, 2.62588e-05, 9.99974e-01],
        [2.32911e-17, 1.06462e-05, 9.99989e-01],
        [6.55976e-23, 6.56065e-04, 9.99344e-01],
        [1.56611e-11, 1.07898e-04, 9.99892e-01],
        [6.55976e-23, 6.56065e-04, 9.99344e-01],
        [1.69187e-17, 4.25846e-07, 1.00000e+00],
        [6.55976e-23, 6.56065e-04, 9.99344e-01],
        [3.20557e-17, 2.66091e-04, 9.99734e-01],
        [1.13774e-11, 4.31629e-06, 9.99996e-01],
        [1.56611e-11, 1.07898e-04, 9.99892e-01],
        [1.77794e-28, 3.89072e-02, 9.61093e-01],
        [6.55976e-23, 6.56065e-04, 9.99344e-01],
        [4.38498e-17, 6.61015e-03, 9.93390e-01],
        [1.26571e-28, 5.03000e-01, 4.97000e-01],
        [1.26571e-28, 5.03000e-01, 4.97000e-01],
        [1.77794e-28, 3.89072e-02, 9.61093e-01],
        [2.32911e-17, 1.06462e-05, 9.99989e-01],
        [4.76799e-23